#  Setup stuff: Connectivity

In [ ]:
#  This file also includes;  at_version()

# import os

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")

In [ ]:
#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)

In [ ]:
#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)

In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

#  Setup stuff: Debugging

In [ ]:
#  LIST ALL GRAPHS

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "     DB GRAPH: ", l_graph.name, "    GRAPH VERSION: ", l_graph.version)

print("--")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
my_graph.query("CALL graph.schema() RETURN *")


In [ ]:
my_client.widgets().operations()


In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


In [ ]:
# ##################################################################

In [ ]:
#  Point to a prior version of our graph ..
#
#     Future, also later in time; add Client.transaction()

l_version = 5

my_graph2 = my_graph.at_version(l_version)

print("--")


In [ ]:
if 'my_graph2' in globals():
   display(my_graph2.num_nodes())
   display(my_graph2.num_edges())
else: 
   print("--")


# Create: Vertices/nodes, edges ..

In [ ]:
import pandas as pd

print("--")


In [ ]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

df_airports.head(20)


In [ ]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

df_flights.head(30)


In [9]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

print("--")


          0/? [?op/s]

CancelledError: 

In [ ]:

#  What's in the vertices/nodes ..

l_result = my_graph.query("""

   MATCH ( n ) 
   RETURN n
   
   """)

print(l_result[0:30])


In [ ]:
#  What's in the edge .. 

l_result = my_graph.query("""

   MATCH ( n )  - [r] ->  (m)
   RETURN n.id AS FROM, m.id AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS
   
   """)

print(l_result[0:30])


In [ ]:
# ##################################################################

In [ ]:

#  Brian's logic thing ..

l_result1 = my_graph.query("""

   MATCH ( n ) 
   RETURN COUNT(*)
   
   """)

display(l_result1)


l_result2 = my_graph.query("""

   MATCH ( n ) 
   WHERE
      (n.id <> 'ORD' OR n.id <> 'MKE')
   
   RETURN COUNT(*)
   
   """)

display(l_result2)


#  Administration/testing stuff ..

In [ ]:
#  Save current graph as an RDG ..

from katana.remote import export_data

l_graph_path = "gs://farrell-bucket/sssp/farrell-sssp"

export_data.rdg(my_graph, l_graph_path)

print("--")


In [ ]:
#  Load from RDG ..
#     Have an empty graph created already ..

from katana.remote import import_data


l_graph_path = "gs://farrell-bucket/sssp/farrell-sssp"

import_data.rdg(my_graph, l_graph_path)

print("--")
